# Solutions to Exercise 1

## Financial Analytics

### UChicago ADSP

#### Spring 2024

* Mark Hendricks
* hendricks@uchicago.edu

#### This assignment is not submitted, graded, or for credit. 
Rather, it is assigned for learning purposes.

***

# 1. Calculating Returns

Use the data in `data/equity_data.xlsx`
* tab `prices AAPL`: columns `Unadjusted Price` and `Adjusted Price`
* tab  `dividends AAPL`: columns `ex_date` and `dividend_amount`

### 1.1
For the most recent 2 dividend dates, calculate the one-day return to AAPL using the unadjusted price and the dividend amount. 

That is, for a given dividend, calculate the return for the day prior to the "ex date" and ending on the "ex date". Do this for at least the most recent two dividends. (Though it is probably just as easy to adjust the return on every dividend date.)

How close is this to the return calculated from the adjusted price percentage change?

### 1.2
For the most recent stock split, (see multiple in `dividend_amount` and indicated in the column `dividend_type`,) calculate the return to AAPL using the unadjusted price along with this split and dividend info.

How close is this computed return to that calculated from the adjusted price growth?

***

# 2. Stock Sensitivity

Use the data in `data/equity_data.xlsx`, in the tab `etf history`.

For the ETFs listed in `etf history`, calculate their percentage change over time. (These are adjusted prices, so this is a measure of their returns.)

## 2.1
Report the correlation table.

## 2.2

In equities, there is an important metric: a stock's **beta**:

$$r_t = \alpha + \beta r^{\text{SPY}}_t + \epsilon_t$$

This beta is analyzed with respect to many factors, but it is most widely analyzed with respect to the S&P 500 (or a similar, broad equity index. Thus the notation $r^{\text{SPY}}_t$ on the right-hand side.

### Calculation
For each ETF return series, estimate the regression above. (You will estimate many regressions, in a loop, each using SPY as the right-hand-side factor.)

Report these betas. Which ETF has the most overall stock-market risk?

#### Hint: 

To estimate the regression, consider using the following approach:

`from sklearn.linear_model import LinearRegression`

`LinearRegression().fit(X,y).coef_`

***

# <span style="color:red">Solutions</span>

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)
import sys
sys.path.insert(0, '../cmds')
from portfolio import *

## <span style="color:red">Solution 1</span>

In [2]:
LOADFILE = '../data/equity_data.xlsx'
TICK = 'AAPL'
TAB = f'prices {TICK}'
px = pd.read_excel(LOADFILE,sheet_name=TAB).set_index('date')

TAB = f'dividends {TICK}'
dvds = pd.read_excel(LOADFILE,sheet_name=TAB).set_index('ex_date')
dvds

,record_date,declared_date,payable_date,dividend_amount,dividend_frequency,dividend_type
ex_date,,,,,,
2024-02-09,2024-02-12,2024-02-01,2024-02-15,0.240000,Quarter,Regular Cash
2023-11-10,2023-11-13,2023-11-02,2023-11-16,0.240000,Quarter,Regular Cash
2023-08-11,2023-08-14,2023-08-03,2023-08-17,0.240000,Quarter,Regular Cash
2023-05-12,2023-05-15,2023-05-04,2023-05-18,0.240000,Quarter,Regular Cash
2023-02-10,2023-02-13,2023-02-02,2023-02-16,0.230000,Quarter,Regular Cash
...,...,...,...,...,...,...
1988-02-12,1988-02-19,1988-01-28,1988-03-15,0.000714,Quarter,Regular Cash
1987-11-17,1987-11-23,1987-11-12,1987-12-15,0.000714,Quarter,Regular Cash
1987-08-10,1987-08-14,1987-07-31,1987-09-15,0.000536,Quarter,Regular Cash


### 1.1

With the adjustments, returns are identical up to 4 decimal places.

In [3]:
rets = px.pct_change()
rets['Unadjusted Price Modified'] = rets.copy()['Unadjusted Price']

for dt in dvds.index:
    if dt in rets.index:
        if dvds.loc[dt,'dividend_type']=='Regular Cash':
            rets.loc[dt,'Unadjusted Price Modified'] = (px.loc[dt,'Unadjusted Price'] + dvds.loc[dt,'dividend_amount']) / px['Unadjusted Price'].shift().loc[dt] - 1

        elif dvds.loc[dt,'dividend_type']=='Stock Split':
            rets.loc[dt,'Unadjusted Price Modified'] = dvds.loc[dt,'dividend_amount'] * px.loc[dt,'Unadjusted Price'] / px['Unadjusted Price'].shift().loc[dt] - 1

In [4]:
rets.loc[dvds.index[0:4],:].style.format('{:.4%}')

,Adjusted Price,Unadjusted Price,Unadjusted Price Modified
ex_date,,,
2024-02-09 00:00:00,0.4094%,0.2814%,0.4089%
2023-11-10 00:00:00,2.3222%,2.1874%,2.3190%
2023-08-11 00:00:00,0.0338%,-0.1011%,0.0337%
2023-05-12 00:00:00,-0.5416%,-0.6791%,-0.5410%


### 1.2

By adjusting for the split, we get almost the exact same return as the "Adjusted Price" series.

Note that the "Unadjusted Price" would show a loss of nearly 75%!

In [5]:
rets.loc[dvds[dvds['dividend_type']=='Stock Split'].index[0]].to_frame().style.format('{:.4%}')

,2020-08-31 00:00:00
Adjusted Price,3.3916%
Unadjusted Price,-74.1522%
Unadjusted Price Modified,3.3912%


### Extra Analysis

#### Correlation

The unadjusted price series return has relatively low correlation with the adjusted series. 

However, after the modifications made above, it is over 99% correlated.

In [6]:
rets.corr().style.format('{:.1%}')

,Adjusted Price,Unadjusted Price,Unadjusted Price Modified
Adjusted Price,100.0%,75.2%,100.0%
Unadjusted Price,75.2%,100.0%,75.3%
Unadjusted Price Modified,100.0%,75.3%,100.0%


#### Performance

Without the adjustment, the stock splits and unaccounted dividends lead to a much lower mean and higher volatility.

In [7]:
performanceMetrics(rets,annualization=252).style.format('{:.1%}')

,Mean,Vol,Sharpe,Min,Max
Adjusted Price,36.6%,32.9%,111.1%,-17.9%,13.9%
Unadjusted Price,25.1%,42.9%,58.5%,-85.5%,13.9%
Unadjusted Price Modified,36.0%,32.9%,109.3%,-17.9%,13.9%


***

# <span style="color:red">Solution 2</span>

In [8]:
LOADFILE = '../data/equity_data.xlsx'
TAB = 'etf history'
etfs = pd.read_excel(LOADFILE,sheet_name=TAB).set_index('date')
spy = etfs[['SPY']]

## <span style="color:red">2.1</span>

In [9]:
rets = etfs.pct_change(fill_method=None)
rets.corr().style.format('{:.1%}').background_gradient(cmap='coolwarm')

,SPY,UPRO,EEM,VGK,EWJ,IYR,DBC,HYG,TIP,BITO
SPY,100.0%,99.9%,82.1%,86.7%,64.3%,71.6%,41.3%,68.3%,-11.4%,44.5%
UPRO,99.9%,100.0%,79.4%,84.3%,72.5%,77.3%,41.8%,76.3%,-7.9%,44.4%
EEM,82.1%,79.4%,100.0%,83.4%,74.0%,66.3%,44.9%,59.9%,-9.5%,37.7%
VGK,86.7%,84.3%,83.4%,100.0%,76.3%,68.5%,46.6%,64.6%,-7.8%,38.4%
EWJ,64.3%,72.5%,74.0%,76.3%,100.0%,51.8%,35.5%,55.4%,-5.1%,32.7%
IYR,71.6%,77.3%,66.3%,68.5%,51.8%,100.0%,28.6%,53.7%,-4.0%,33.6%
DBC,41.3%,41.8%,44.9%,46.6%,35.5%,28.6%,100.0%,36.8%,6.4%,12.1%
HYG,68.3%,76.3%,59.9%,64.6%,55.4%,53.7%,36.8%,100.0%,7.1%,36.9%
TIP,-11.4%,-7.9%,-9.5%,-7.8%,-5.1%,-4.0%,6.4%,7.1%,100.0%,10.8%
BITO,44.5%,44.4%,37.7%,38.4%,32.7%,33.6%,12.1%,36.9%,10.8%,100.0%


## <span style="color:red">2.2</span>

In [10]:
x = spy.pct_change(fill_method=None).dropna()

betas = pd.DataFrame(index=etfs.columns,dtype=float, columns=['SPY beta'])
for col in etfs.columns:
    y = etfs[[col]].pct_change(fill_method=None).dropna()
    y, x = y.align(x,join='inner',axis=0)
    betas.loc[col] = LinearRegression().fit(x,y).coef_

betas.T.style.format('{:.2f}')

,SPY,UPRO,EEM,VGK,EWJ,IYR,DBC,HYG,TIP,BITO
SPY beta,1.00,3.00,1.02,1.05,0.74,0.94,0.43,0.39,-0.03,1.43
